Reference: https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/mapgccv/miscellaneous/sudoku.ipynb#scrollTo=jxi-HR24j7jh

In [1]:
%pip install -q amplpy ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.7 MB/s eta 0:00:00


In [2]:
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://colab.ampl.com).


In [3]:
%%ampl_eval
reset;
option solver highs;
# The base number of this sudoku; 3 is the default (9 numbers game)
param BASE default 3;
# The line/column lenght, derived from BASE
param L := BASE*BASE;

# Set of all Rows
set ROWS := {1..L};
# Set of all columns
set COLS := {1..L};
# This indexed set memorizes the tuples of coordinates for each
# sub-square making up the grid
set SUBSQUARES{sr in 1..BASE, sc in 1..BASE} within {ROWS, COLS}
	            = {(sr-1)*BASE+1..sr*BASE, (sc-1)*BASE+1..sc*BASE};

In [4]:
ampl.display("SUBSQUARES")

set SUBSQUARES[1,1] := (1,1) (1,2) (1,3) (2,1) (2,2) (2,3) (3,1) (3,2) (3,3);
set SUBSQUARES[1,2] := (1,4) (1,5) (1,6) (2,4) (2,5) (2,6) (3,4) (3,5) (3,6);
set SUBSQUARES[1,3] := (1,7) (1,8) (1,9) (2,7) (2,8) (2,9) (3,7) (3,8) (3,9);
set SUBSQUARES[2,1] := (4,1) (4,2) (4,3) (5,1) (5,2) (5,3) (6,1) (6,2) (6,3);
set SUBSQUARES[2,2] := (4,4) (4,5) (4,6) (5,4) (5,5) (5,6) (6,4) (6,5) (6,6);
set SUBSQUARES[2,3] := (4,7) (4,8) (4,9) (5,7) (5,8) (5,9) (6,7) (6,8) (6,9);
set SUBSQUARES[3,1] := (7,1) (7,2) (7,3) (8,1) (8,2) (8,3) (9,1) (9,2) (9,3);
set SUBSQUARES[3,2] := (7,4) (7,5) (7,6) (8,4) (8,5) (8,6) (9,4) (9,5) (9,6);
set SUBSQUARES[3,3] := (7,7) (7,8) (7,9) (8,7) (8,8) (8,9) (9,7) (9,8) (9,9);



In [5]:
%%ampl_eval
# The variables representing the numbers at all positions
var x{ROWS, COLS} >=1, <=L integer;

var IsN{1..L, COLS, ROWS} binary;

# Set this parameter to non-zero to force a position to have that value
param givenData{ROWS, COLS} default 0;

# Dummy objective, just to "encourage" the solver to get the same
# objective function in case of a degenerate sudoku
maximize z: x[1,1];

# Fix input data
subject to fixGivenData {r in ROWS, c in COLS : givenData[r,c] > 0}:
  x[r,c] = givenData[r,c];

# Each position only one number
subject to MIPOnlyOneNumber {r in ROWS, c in COLS}:
  sum{n in 1..L} IsN[n,c,r] = 1;

# Each number must be present in each row once
subject to MIPEachRowOneNumber {r in ROWS, n in 1..L}:
  sum{c in COLS} IsN[n,c,r] = 1;

# Each number must be present in each col once
subject to MIPEachColOneNumber {r in COLS, n in 1..L}:
  sum{c in ROWS} IsN[n,c,r] = 1;

# Each number must be present in each subsquare once
subject to MIPEachSquareOneNumber {n in 1..L, sr in 1..BASE, sc in 1..BASE}:
  sum{(r, c) in SUBSQUARES[sr, sc]} IsN[n, c, r] = 1;

# Link to the logical model variable
subject to MIPLinkToX {r in ROWS, c in COLS}:
  sum{n in 1..L} IsN[n,c,r]*n =x[r,c];

# Define a named problem to quickly switch between formulations
problem sudokuMIP: x, IsN, z, fixGivenData, MIPOnlyOneNumber, MIPEachRowOneNumber, MIPEachColOneNumber, MIPEachSquareOneNumber, MIPLinkToX;

In [6]:
from random import seed, random

BASE = 3
seed(1234)


def random_state():
    ampl.param["BASE"] = BASE
    # ampl.param["givenData"] = {(1, 1): 0}
    if BASE != 3:
        return
    solution = [
        [2, 5, 7, 8, 6, 3, 1, 4, 9],
        [4, 9, 6, 5, 7, 1, 8, 3, 2],
        [8, 1, 3, 9, 4, 2, 7, 6, 5],
        [1, 6, 5, 2, 9, 4, 3, 7, 8],
        [9, 8, 4, 1, 3, 7, 5, 2, 6],
        [3, 7, 2, 6, 5, 8, 4, 9, 1],
        [7, 2, 9, 4, 8, 5, 6, 1, 3],
        [5, 3, 1, 7, 2, 6, 9, 8, 4],
        [6, 4, 8, 3, 1, 9, 2, 5, 7],
    ]
    ampl.param["givenData"] = {
        (i + 1, j + 1): solution[i][j] if random() <= 1 / 3.0 else 0
        for i in range(9)
        for j in range(9)
    }


random_state()

In [7]:
ampl.display("givenData")

givenData [*,*]
:   1   2   3   4   5   6   7   8   9    :=
1   0   0   7   0   0   0   0   4   0
2   4   9   0   0   0   0   8   3   2
3   8   0   0   9   0   0   0   6   0
4   1   0   0   0   0   4   0   0   0
5   9   0   4   1   0   0   5   0   6
6   0   7   0   0   0   8   4   9   1
7   0   2   9   0   8   0   6   0   0
8   0   0   0   0   0   0   0   0   0
9   0   4   8   3   0   9   0   0   0
;



In [8]:
problem_name = "sudokuMIP"
ampl.eval(f"solve {problem_name};")

HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 6
194 simplex iterations
1 branching nodes
 


In [9]:
# ampl.get_data("x").to_dict()
ampl.display("x")

x [*,*]
:   1   2   3   4   5   6   7   8   9    :=
1   6   2   7   3   8   1   5   4   9
2   4   9   1   7   6   5   8   3   2
3   8   5   3   9   2   4   1   6   7
4   1   3   5   9   6   4   8   2   7
5   9   8   4   1   7   2   5   3   6
6   2   7   6   5   3   8   4   9   1
7   7   2   9   1   8   5   6   3   4
8   3   5   1   7   6   4   2   8   9
9   6   4   8   3   2   9   1   5   7
;

